In [1]:
import sys
sys.path.append('..')

import os
import time
import numpy as np
import torch
from datasets.ice import IceWithProposals
from torch.utils.data import DataLoader
from torch import nn
from models.basic_pga.utils import get_image_dicts, build_pos_tensors, build_rand_inds
from models.basic_axial.basic_axial_parts import BlockAxial
from models.basic_pga.basic_pga_parts import BlockPGA


# def conv1x1(in_planes, out_planes, stride=1):
#     return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


# class BlockPGA(nn.Module):
#     def __init__(self, channels, embedding_dims, img_shape=(300, 300)):
#         super(BlockPGA, self).__init__()
#         self.channels = channels
#         self.embedding_dims = embedding_dims
#         self.embedding_dims_double = embedding_dims * 2
#         self.img_shape = img_shape

#         self.conv1 = conv1x1(self.channels, self.embedding_dims)
#         self.bn1 = nn.BatchNorm2d(self.embedding_dims)
#         self.relu = nn.ReLU(inplace=True)

#         self.attn = PropAttention(dim=self.embedding_dims, heads=2, img_crop=img_shape[0])

#         self.conv2 = conv1x1(self.embedding_dims_double, self.embedding_dims)
#         self.bn2 = nn.BatchNorm2d(self.embedding_dims)

#     def forward(self, x, obj_dict, bg_dict):
#         # print(x.shape)
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)

#         x_attn = self.attn(x, obj_dict, bg_dict)
#         x_attn = self.relu(x_attn)

#         x = torch.cat((x_attn, x), dim=1)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu(x)

#         return x


# class PropAttention(nn.Module):
#     def __init__(self, dim, heads, img_crop, dim_heads=None):
#         assert (dim % heads) == 0, 'hidden dimension must be divisible by number of heads'
#         super().__init__()
#         self.dim_heads = (dim // heads) if dim_heads is None else dim_heads

#         inds = build_rand_inds(heads, img_crop)
#         inds_tensor = torch.LongTensor(inds)
#         self.rand_inds = nn.Parameter(inds_tensor, requires_grad=False)

#         self.heads = heads
#         self.img_crop = img_crop
#         self.to_q = nn.Linear(self.dim_heads, self.dim_heads, bias=False)
#         self.to_kv = nn.Linear(self.dim_heads, 2 * self.dim_heads, bias=False)
#         self.to_out = nn.Linear(dim, dim)

#     def construct(self, t, obj_dict, bg_dict, inds):
#         img_flat = t.view(-1, self.img_crop ** 2)

#         obj_vecs = []
#         bg_vecs = []
#         num_obj = self.img_crop // 2
#         num_bg = self.img_crop - num_obj
#         for i in range(num_obj):
#             obj_vecs.append(build_pos_tensors(img_flat, obj_dict, inds[i]))
#         for i in range(num_bg):
#             bg_vecs.append(build_pos_tensors(img_flat, bg_dict, inds[i + num_obj]))

#         obj_tensor = torch.stack(obj_vecs, dim=0)
#         bg_tensor = torch.stack(bg_vecs, dim=0)

#         all_tensor = torch.cat((obj_tensor, bg_tensor), dim=0)

#         return all_tensor

#     def destruct(self, t, img, obj_dict, bg_dict, inds):
#         ts = torch.chunk(t, 2, dim=0)

#         obj_tensor_tuple = torch.chunk(ts[0], ts[0].shape[0], dim=0)
#         bg_tensor_tuple = torch.chunk(ts[1], ts[1].shape[0], dim=0)

#         for i, vec in enumerate(obj_tensor_tuple):
#             img = self.add_vec_to_tensor(img, inds[i], vec.squeeze(0).transpose(1, 0), obj_dict)
#         for i, vec in enumerate(bg_tensor_tuple):
#             img = self.add_vec_to_tensor(img, inds[i + len(obj_tensor_tuple)], vec.squeeze(0).transpose(1, 0), bg_dict)

#         return img

#     def add_vec_to_tensor(self, t, inds, vec, obj_dict):
#         # for i, v in zip(inds, vec):
#         for i, v in zip(inds.detach().cpu().numpy(), vec):
#             obj_ind = obj_dict[i]
#             x = obj_ind // self.img_crop
#             y = obj_ind % self.img_crop
#             t[:, :, x, y] = v
#         return t

#     def forward(self, x, obj_dict, bg_dict, kv=None):
# #         x = x.clone().detach()
        
#         start_time = time.time()
#         img_heads = torch.chunk(x.cpu(), self.heads, dim=1)
#         head_list = []
#         for inds, h in zip(self.rand_inds, img_heads):
#             head_list.append(self.construct(h, obj_dict, bg_dict, inds))

#         out = torch.cat(head_list, dim=0).view(self.heads * self.img_crop, self.img_crop, -1).cuda()
#         print(f"Constructing finished in {time.time() - start_time} seconds.")
        
#         start_time = time.time()
#         kv = out if kv is None else kv
#         q, k, v = (self.to_q(out), *self.to_kv(kv).chunk(2, dim=-1))
#         dots = torch.einsum('bie,bje->bij', q, k) * (self.dim_heads ** -0.5)
#         dots = dots.softmax(dim=-1)
#         out = torch.einsum('bij,bje->bie', dots, v)

#         out = out.view(self.heads * self.img_crop, -1, self.img_crop).cpu()
#         print(f"Matrix math finished in {time.time() - start_time} seconds.")
        
#         ts = torch.chunk(out, self.heads, dim=0)
        
#         start_time = time.time()
#         new_img_list = []
#         for t, img_head, inds in zip(ts, img_heads, self.rand_inds):
#             new_img_list.append(self.destruct(t, img_head, obj_dict, bg_dict, inds))
#         out_final = torch.cat(new_img_list, dim=1).permute(0, 2, 3, 1).contiguous().cuda()
#         out_final = self.to_out(out_final)
#         print(f"Destructing finished in {time.time() - start_time} seconds.")
        
#         return out_final.permute(0, 3, 1, 2).contiguous()

In [2]:
data_dir = '../data'
img_scale = .35
img_crop = 220

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_set = IceWithProposals(os.path.join(data_dir, 'imgs'), os.path.join(data_dir, 'masks'),
                  os.path.join(data_dir, 'txt_files'), os.path.join(data_dir, 'proposals/binary_250_16'),
                               'train', img_scale, img_crop)
train_loader = DataLoader(train_set, batch_size=1)

In [3]:
start_time = time.time()
batch = train_set[0]
print(f"Loading batch finished in {time.time() - start_time} seconds.")

Loading batch finished in 0.5951581001281738 seconds.


In [4]:
obj_dict, bg_dict = batch['obj_dict'], batch['bg_dict']

In [5]:
x = torch.rand(1,3,img_crop,img_crop).to(device)

In [6]:
block = BlockPGA(3, 10, img_shape=(img_crop, img_crop)).to(device)

In [7]:
start_time = time.time()
out = block(x, obj_dict, bg_dict)
print(f"Finished in {time.time() - start_time} seconds.")

Constructing finished in 1.2000470161437988 seconds.
Matrix math finished in 0.006580829620361328 seconds.
Destructing finished in 2.502077341079712 seconds.
Finished in 4.1572065353393555 seconds.


In [4]:
prop = batch['prop']
prop_flat = prop.flatten()

In [5]:
(prop_flat==0)

tensor([True, True, True,  ..., True, True, True])

In [6]:
inds_bg = (prop_flat == 0).nonzero(as_tuple=True)[0]
inds_obj = (prop_flat == 1).nonzero(as_tuple=True)[0]
print(len(inds_bg), len(inds_obj))

16888 31512


In [51]:
extended_bg = torch.cat((inds_bg, inds_bg), dim=0)
while len(extended_bg) < img_crop**2:
    extended_bg = torch.cat((extended_bg, inds_bg), dim=0)
extended_bg = extended_bg[:img_crop**2]
print(len(extended_bg))

extended_obj = torch.cat((inds_obj, inds_obj), dim=0)
while len(extended_obj) < img_crop**2:
    extended_obj = torch.cat((extended_obj, inds_obj), dim=0)
extended_obj = extended_obj[:img_crop**2]
print(len(extended_obj))

48400
48400


In [52]:
bg_dict = {i: x.item() for i,x in enumerate(extended_bg)}
obj_dict = {i: x.item() for i,x in enumerate(extended_obj)}

# On Cuda

In [6]:
start_time = time.time()
out = block(x, prop)
print(f"Finished in {time.time() - start_time} seconds.")

Getting dicts finished in 9.985198974609375 seconds.
Constructing finished in 1.2408664226531982 seconds.
Matrix math finished in 0.0014796257019042969 seconds.
Destructing finished in 3.336125135421753 seconds.
Finished in 15.011268854141235 seconds.


# On CPU

In [6]:
start_time = time.time()
out = block(x, prop)
print(f"Finished in {time.time() - start_time} seconds.")

Getting dicts finished in 4.533999919891357 seconds.
Constructing finished in 1.207212209701538 seconds.
Matrix math finished in 0.06548714637756348 seconds.
Destructing finished in 2.323352098464966 seconds.
Finished in 8.481204509735107 seconds.


In [5]:
test = torch.randn(1, 20, img_crop, img_crop).to(device)

In [6]:
block_pga = BlockPGA(20, 20, img_shape=(img_crop, img_crop)).to(device)

In [7]:
block_axial = BlockAxial(10, 10, img_shape=(img_crop, img_crop)).to(device)

In [ ]:
out = block_pga(test, obj_dict, bg_dict)

In [12]:
params = 0
for name, param in block_pga.named_parameters():
    if param.requires_grad:
        print(name, param.numel())
        params += param.numel()
print(params)

conv1.weight 100
bn1.weight 10
bn1.bias 10
attn.to_q.weight 25
attn.to_kv.weight 50
attn.to_out.weight 100
attn.to_out.bias 10
conv2.weight 200
bn2.weight 10
bn2.bias 10
525


In [13]:
params = 0
for name, param in block_axial.named_parameters():
    if param.requires_grad:
        print(name, param.numel())
        params += param.numel()
print(params)

conv1.weight 100
bn1.weight 10
bn1.bias 10
attn.axial_attentions.0.fn.to_q.weight 100
attn.axial_attentions.0.fn.to_kv.weight 200
attn.axial_attentions.0.fn.to_out.weight 100
attn.axial_attentions.0.fn.to_out.bias 10
attn.axial_attentions.1.fn.to_q.weight 100
attn.axial_attentions.1.fn.to_kv.weight 200
attn.axial_attentions.1.fn.to_out.weight 100
attn.axial_attentions.1.fn.to_out.bias 10
conv2.weight 200
bn2.weight 10
bn2.bias 10
1160
